In [9]:
import os
import cv2
import torch
import numpy as np
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, models
from concurrent.futures import ThreadPoolExecutor
from torch.utils.data import Dataset, DataLoader
import torch.profiler

# Ensure CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Torch empty cache
torch.cuda.empty_cache()

Using device: cuda


In [ ]:
# Load the model
model_path = "../model/efficientnet_coffee.pth"
model = models.efficientnet_b0(weights=False)
checkpoint = torch.load(model_path, map_location=device)
num_classes = 5
model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=num_classes)
model.load_state_dict(checkpoint, strict=False)

model.to(device)
model.eval()


C:\Users\quanb\AppData\Local\Temp\ipykernel_25380\3590970944.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
target_layer = model.features[-1]
print(target_layer)

In [23]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model.eval().to(device)
        self.target_layer = target_layer.to(device)
        self.activation = None
        self.gradients = None

        self.target_layer.register_forward_hook(self.forward_hook)
        self.target_layer.register_full_backward_hook(self.backward_hook)

    def forward_hook(self, module, input, output):
        self.activation = output.detach()

    def backward_hook(self, module, grad_in, grad_out):
        self.gradients = grad_out[0].detach()

    def generate_heatmap(self, input_tensor, target_class=None):
        input_tensor = input_tensor.to(device)
        output = self.model(input_tensor)

        if target_class is None:
            target_class = output.argmax(dim=1).item()

        self.model.zero_grad()
        output[0, target_class].backward()

        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * self.activation).sum(dim=1, keepdim=True)
        cam = torch.relu(cam).squeeze().cpu().numpy()

        eps = 1e-8
        cam = (cam - cam.min()) / (cam.max() - cam.min() + eps)
        return cam

    def apply_bounding_box(self, original_image, heatmap, threshold):
        h, w, _ = original_image.shape
        heatmap_resized = cv2.resize(heatmap, (w, h))

        # Apply threshold and morphological closing to reduce noise
        _, binary_map = cv2.threshold(heatmap_resized, threshold, 255, cv2.THRESH_BINARY)
        binary_map = binary_map.astype(np.uint8)
        
        # Close small gaps between regions
        kernel = np.ones((3, 3), np.uint8)
        closed_map = cv2.morphologyEx(binary_map, cv2.MORPH_CLOSE, kernel)

        # Find contours in the processed binary map
        contours, _ = cv2.findContours(closed_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        return original_image

In [ ]:
def apply_gradcam(model, target_layer, image_path, threshold):
    try:
        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"Could not read image: {image_path}")
            return None

        input_tensor = transform(Image.fromarray(original_image)).unsqueeze(0).to(device)
        gradcam = GradCAM(model, target_layer)
        heatmap = gradcam.generate_heatmap(input_tensor)
        
        # Optional: Overlay heatmap
        heatmap_overlay = gradcam.overlay_heatmap(original_image, heatmap)
        result_image = gradcam.apply_bounding_box(heatmap_overlay, heatmap, threshold)

        return result_image
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [ ]:
def overlay_heatmap(self, original_image, heatmap, alpha=0.4):
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlaid_image = cv2.addWeighted(original_image, 1 - alpha, heatmap_colored, alpha, 0)
    return overlaid_image

In [ ]:
def dynamic_threshold(heatmap, percentile=75):
    return np.percentile(heatmap, percentile)

In [ ]:
def process_dataset(model, target_layer, root_dir, output_dir, threshold, max_workers=4):
    os.makedirs(output_dir, exist_ok=True)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_path):
                continue

            output_class_dir = os.path.join(output_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            for img_name in os.listdir(class_path):
                if not img_name.startswith("orig_"):
                    continue
                
                img_path = os.path.join(class_path, img_name)
                if not img_path.lower().endswith((".jpg", ".jpeg", ".png")):
                    continue

                future = executor.submit(apply_gradcam, model, target_layer, img_path, threshold)
                futures.append((future, output_class_dir, img_name))

        for future, output_dir, img_name in futures:
            result_image = future.result()
            if result_image is not None:
                output_path = os.path.join(output_dir, img_name)
                cv2.imwrite(output_path, result_image)

In [26]:
threshold_value = 0.5
process_dataset(model, target_layer, f"../data/Final_CLD_data/train", "../data/bounding_box_{threshold_value}_resnet50_1", threshold_value)

AttributeError: 'RemovableHandle' object has no attribute 'to'